In [5]:
import numpy as np

seq1 = "ATTAAAGGTTTA"
seq2 = "CAACTTTCGATC"

scores = np.zeros((len(seq1) + 1, len(seq2) + 1))
scores[:,0] = [-1 * i for i in range(0, len(seq1)+1)]
scores[0,:] = [-1 * i for i in range(0, len(seq2)+1)]

In [6]:
# implement needleman-wunsch

pointers = {}
indel_score = -1
match_score = 1

for i in range(1, len(seq1) * 2):
    row_sel = i
    col_sel = 1
    while row_sel >= 1 and col_sel < len(seq2)+1:
        if row_sel >= len(seq1)+1:
            row_sel -= 1
            col_sel += 1
            continue
        diag = scores[row_sel-1,col_sel-1]
        left = scores[row_sel, col_sel-1] - 1
        top = scores[row_sel-1, col_sel] - 1

        if seq1[row_sel-1] == seq2[col_sel-1]:
            diag += 1
        else:
            diag -= 1

        combined = [diag, left, top]
        max_combined = max(combined)
        ind = [i for i, j in enumerate(combined) if j == max_combined]

        pointer_key = "{}_{}".format(row_sel, col_sel)
        pointers[pointer_key] = []
        if 0 in ind:
            pointers[pointer_key].append((row_sel-1,col_sel-1))
        if 1 in ind:
            pointers[pointer_key].append((row_sel, col_sel-1))
        if 2 in ind:
            pointers[pointer_key].append((row_sel-1, col_sel))

        scores[row_sel, col_sel] = max_combined
        row_sel -= 1
        col_sel += 1

In [7]:
alignments = []

def parse_alignments(row, col, align1, align2):
    pointer_index = "{}_{}".format(row, col)
    if row==1 and col==1:
        alignments.append([align1, align2])
        return
    if pointer_index not in pointers:
        return

    for pointer in pointers[pointer_index]:
        if pointer[0] == row:
            parse_alignments(pointer[0], pointer[1], "_" +  align1, seq2[col-1] + align2)
        elif pointer[1] == col:
            parse_alignments(pointer[0], pointer[1], seq1[row-1] + align1, "_" + align2)
        else:
            parse_alignments(pointer[0], pointer[1], seq1[row-1] + align1, seq2[col-1] + align2)

In [8]:
parse_alignments(scores.shape[0]-1, scores.shape[1]-1, "", "")
alignments

[['TTAAAGGTTT__A__', '___AA_CTTTCGATC'],
 ['TTAAAGGTTT__A__', '__A_A_CTTTCGATC'],
 ['TTAAAGGTTT__A__', '__AA__CTTTCGATC'],
 ['TTAAAGGTTT__A__', '___AAC_TTTCGATC'],
 ['TTAAAGGTTT__A__', '__A_AC_TTTCGATC'],
 ['TTAAAGGTTT__A__', '__AA_C_TTTCGATC'],
 ['TTAAAGGTTT__A__', '__AAC__TTTCGATC']]

In [9]:
print(alignments[0][0])
print(alignments[0][1])

TTAAAGGTTT__A__
___AA_CTTTCGATC
